In [1]:
import qiskit
import numpy as np

from PatchedMeasCal.tensor_patch_cal import TensorPatchFitter
from PatchedMeasCal.inv_measure_methods import aim, sim
from PatchedMeasCal.jigsaw import jigsaw
from PatchedMeasCal.qiskit_meas_fitters import qiskit_full, qiskit_linear


from PatchedMeasCal.fake_backends import LocalSimulator


from PatchedMeasCal import state_prep_circuits
from PatchedMeasCal.fake_measurement_distributions import renormalise_measurement_results
from PatchedMeasCal.utils import Progressbar

from PatchedMeasCal.fake_backends import Grid, Hexagonal16, FullyConnected

from PatchedMeasCal.state_prep_circuits import GHZ_prep, GHZ_state_dist
from PatchedMeasCal.bv import bv_circuit_cmap



from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter
from qiskit.ignis.mitigation.measurement import tensored_meas_cal, TensoredMeasFitter

from functools import partial

import qiskit.tools.jupyter

import mthree

import random

qiskit.IBMQ.load_account()
provider = qiskit.IBMQ.get_provider()

%qiskit_job_watcher

Accordion(children=(VBox(layout=Layout(max_width='710px', min_width='710px')),), layout=Layout(max_height='500…

<IPython.core.display.Javascript object>

In [2]:
def distance_measure(x, bv_value):
    if bv_value in x:
        return 1 - (x[bv_value] / sum(x.values()))
    return 1

## Setup
The idea here is that each approachs gets a maximum of 32000 measurement shots to spend
- Backend style methods will spend 50% of that on the build, 50% of that on the run
- Circuit style methods will spend 50% on their full run, and the other 50% on other circuits that need to be exectued
- AIM will divide theirs up evenly

In [3]:
n_qubits = 5
n_meas_shots = 16000
n_build_shots = 16000
n_shots_qiskit_full = n_build_shots // (2 ** n_qubits) 
n_shots_qiskit_partial = n_build_shots // (n_qubits)


n_circuit_method_shots = n_meas_shots + n_build_shots
backend = provider.get_backend('ibmq_quito')

circuit = GHZ_prep(backend)

n_shots_cmc = n_build_shots // (2 * len(backend.configuration().coupling_map))

err_cmap = [[3, 4], [0, 4], [1, 4], [2, 4]]

## Mitigation Methods Build

In [4]:
full_filter = qiskit_full(backend, n_qubits, n_shots_qiskit_full)

# n Circuits to execute
linear_filter = qiskit_linear(backend, n_qubits, n_shots_qiskit_partial)

mit = mthree.M3Mitigation(backend)
mit.cals_from_system(list(range(n_qubits)), n_build_shots // 2)

tpf_err = TensorPatchFitter(backend, n_shots=n_shots_cmc, coupling_map=err_cmap)
tpf_err.build(verbose=True)

tpf = TensorPatchFitter(backend, n_shots=n_shots_cmc)
tpf.build(verbose=True)

ERROR:websocket:error from callback <bound method BaseWebsocketClient.on_close of <qiskit.providers.ibmq.api.clients.websocket.WebsocketClient object at 0x7f39a135eca0>>: on_close() missing 2 required positional arguments: 'status_code' and 'msg'
ERROR:websocket:error from callback <bound method BaseWebsocketClient.on_close of <qiskit.providers.ibmq.api.clients.websocket.WebsocketClient object at 0x7f39a135eca0>>: on_close() missing 2 required positional arguments: 'status_code' and 'msg'
ERROR:websocket:error from callback <bound method BaseWebsocketClient.on_close of <qiskit.providers.ibmq.api.clients.websocket.WebsocketClient object at 0x7f39a135eca0>>: on_close() missing 2 required positional arguments: 'status_code' and 'msg'
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002
ERROR:websocket:close status: 4002


Building Coupling Graph
Building Edge Calibrations
	Building Calibration Circuits
	Building Edge Patches : [====================] 100.0%  ETA: 0s
	Building Edge Patches : [====================] 100.0%  ETA: 0s
	Constructing Calibration Circuits : [====================] 100.0%  ETA: 0s
	Executing Calibration Circuits


ERROR:websocket:close status: 4002


	De-hexing Measurement Results
	Tracing Patched Calibration Results : [====================] 100.0%  ETA: 0s
Building Patch Calibrations
	Merging Patches : [====================] 100.0%  ETA: 0s
Building Measure Fitter
	Building Meas Fitters from Patches : [====================] 100.0%  ETA: 0s
Building Coupling Graph
Building Edge Calibrations
	Building Calibration Circuits
	Building Edge Patches : [====================] 100.0%  ETA: 0s
	Building Edge Patches : [====================] 100.0%  ETA: 0s
	Constructing Calibration Circuits : [====================] 100.0%  ETA: 0s
	Executing Calibration Circuits


ERROR:websocket:close status: 4002


	De-hexing Measurement Results
	Tracing Patched Calibration Results : [====================] 100.0%  ETA: 0s
Building Patch Calibrations
	Merging Patches : [====================] 100.0%  ETA: 0s
Building Measure Fitter
	Building Meas Fitters from Patches : [====================] 100.0%  ETA: 0s


## Run with repetitions
If everything above is working, let's crank it out a few times

In [12]:
backend = provider.get_backend('ibmq_quito')

n_reps = 3
results = {
    'bare':[],
    'full':[],
    'linear':[],
    'aim':[],
    'sim':[],
    'jigsaw':[],
    'cmc':[],
    'cmc_err':[],
    'm3':[]
}

for _ in range(n_reps):

    target_value = random.randint(0, 2 ** (n_qubits - 1) - 1)
    bv_value = bin((target_value << 1) + 1)[2:].zfill(n_qubits)[::-1]
    bv_string = bin(target_value)[2:].zfill(n_qubits - 1)
    circuit = bv_circuit_cmap(bv_string, n_qubits, backend)
    dist_measure = lambda x: distance_measure(x, bv_value)
    
    bare_result_job = qiskit.execute(circuit, 
                         backend, 
                         shots=n_meas_shots, 
                         optimization_level=0,
                         initial_layout=list(range(n_qubits))
                        )
    
    
    bare_result = bare_result_job.result().get_counts()
    
    results['bare'].append(
        dist_measure(bare_result)
    )
    results['full'].append(
        dist_measure(full_filter.apply(bare_result))
    )
    results['linear'].append(
        dist_measure(linear_filter.apply(bare_result))
    )
    results['cmc'].append(
        dist_measure(tpf.apply(bare_result))
    )
    results['cmc_err'].append(
        dist_measure(tpf_err.apply(bare_result))
    )
    
    results ['m3'].append(
        dist_measure(mit.apply_correction(bare_result, list(range(n_qubits))).nearest_probability_distribution())
    )
    
    results['aim'].append(dist_measure(aim(circuit, backend, n_qubits, n_shots=n_circuit_method_shots, equal_shot_distribution=True)))
    results['sim'].append(dist_measure(sim(circuit, backend, n_qubits, n_shots=n_circuit_method_shots, equal_shot_distribution=True)))
    results['jigsaw'].append(dist_measure(jigsaw(circuit, backend, n_circuit_method_shots, equal_shot_distribution=True)))
    

ERROR:websocket:error from callback <bound method BaseWebsocketClient.on_close of <qiskit.providers.ibmq.api.clients.websocket.WebsocketClient object at 0x7f39a09b8c70>>: on_close() missing 2 required positional arguments: 'status_code' and 'msg'
ERROR:websocket:error from callback <bound method BaseWebsocketClient.on_close of <qiskit.providers.ibmq.api.clients.websocket.WebsocketClient object at 0x7f39a09b8c70>>: on_close() missing 2 required positional arguments: 'status_code' and 'msg'
ERROR:websocket:error from callback <bound method BaseWebsocketClient.on_close of <qiskit.providers.ibmq.api.clients.websocket.WebsocketClient object at 0x7f39a09b8c70>>: on_close() missing 2 required positional arguments: 'status_code' and 'msg'
ERROR:websocket:error from callback <bound method BaseWebsocketClient.on_close of <qiskit.providers.ibmq.api.clients.websocket.WebsocketClient object at 0x7f39a09b8c70>>: on_close() missing 2 required positional arguments: 'status_code' and 'msg'
ERROR:websoc

In [13]:
results

{'bare': [0.4183125, 0.48475, 0.358375],
 'full': [0.02496944474226559, 0.1225244501996261, 0.16955610212559058],
 'linear': [0.07527372892606266, 0.1719878544995166, 0.18877842357342622],
 'aim': [0.2476666666666667, 0.3214583333333333, 0.3327083333333334],
 'sim': [0.35446875, 0.40240624999999997, 0.40421874999999996],
 'jigsaw': [0.22136622708111964, 0.27970136318390326, 0.13110988537535373],
 'cmc': [0.2778987206832786, 0.3340514818830528, 0.15519211756501583],
 'cmc_err': [0.29523253824051043, 0.35045320917246936, 0.17302963305647479],
 'm3': [0.06373967257505775, 0.1446703799823844, 0.17495802704873575]}

In [ ]:
bv_quito_results = {'bare': [0.25956250000000003,
  0.265875,
  0.262625,
  0.26937500000000003,
  0.2646875,
  0.26387499999999997,
  0.2573125,
  0.249875,
  0.27362499999999995,
  0.2636875],
 'full': [0.029775313464926623,
  0.06647910744451674,
  0.04667775226332799,
  0.0332809168928796,
  0.05201336090306968,
  0.02816636396648242,
  0.042579175033529204,
  0.044728550884244545,
  0.04091080425915061,
  0.041340332580012795],
 'linear': [0.06685398871637849,
  0.10307654542937617,
  0.08341716484628997,
  0.0682755370919717,
  0.08884475703082517,
  0.061451295852230636,
  0.07950304239046785,
  0.08032007572549499,
  0.07567339834293518,
  0.07817281690779665],
 'aim': [0.27145833333333336,
  0.268125,
  0.26879166666666665,
  0.26529166666666665,
  0.26316666666666666,
  0.2535833333333333,
  0.25654166666666667,
  0.2699166666666667,
  0.26087499999999997,
  0.2625],
 'sim': [0.27584375,
  0.2750625,
  0.27134375,
  0.26821875,
  0.26899999999999996,
  0.26999999999999996,
  0.2675625,
  0.2629375,
  0.27165625,
  0.28025],
 'jigsaw': [0.20800916343216552,
  0.21512919397038882,
  0.25229220293853327,
  0.23102214725988712,
  0.24311394040792333,
  0.2042409427714018,
  0.166963203629511,
  0.16949084162224526,
  0.1937579000713494,
  0.22623957846867676],
 'cmc': [0.15388526001646624,
  0.16621548127143265,
  0.1570145400187437,
  0.16281618433360506,
  0.15862432406425914,
  0.1545232415778796,
  0.15268780422335748,
  0.14605907920957362,
  0.1701612808123521,
  0.15924838384240414], 
 'cmc_err':[0.2744379333219121,
  0.1695567970415347,
  0.16489292681487588,
  0.14460394095542106,
  0.15745310780670307,
  0.15711038837488828,
  0.1574260355967118,
  0.1669498632895472,
  0.17499063777335538]}

In [ ]:
res = bv_quito_results
for r in res:
    avg = np.mean(res[r])
    h_bound = np.max(res[r]) - avg
    l_bound = avg - np.min(res[r])
    print(r, " & ", "$", "%.2f" % avg, "\substack{+", "%.2f" % h_bound, " \\\\ -", "%.2f" %l_bound, "}$", sep='')